In [ ]:
#!import config/Settings.cs
#!import banking/ContaCorrente.cs 
#!import banking/Boleto.cs 
#!import banking/BankFunctions.cs 
#!import toolcalling/ToolCalling.cs  

// Import Azure OpenAI
#r "nuget: Azure.AI.OpenAI, 2.0.0"
using OpenAI.Chat;
var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();


var endpoint = new Uri(azureEndpoint);
var credentials = new System.ClientModel.ApiKeyCredential(apiKey);
var azureClient = new Azure.AI.OpenAI.AzureOpenAIClient(endpoint, credentials);
var chatClient = azureClient.GetChatClient("gpt-4o");

In [ ]:
var system = 
""" 
Voce é um assistente bancário que ajuda um cliente a transacionar em sua conta. Seja solicito e educado.
""";

In [ ]:
var messages = new List<OpenAI.Chat.ChatMessage>();

messages.Add(new OpenAI.Chat.SystemChatMessage(system))

In [ ]:
ToolCalling caller = new ToolCalling();

var contaCorrentPrincipal = new ContaCorrente("123");
contaCorrentPrincipal.Depositar(1000);

Dictionary<string, ContaCorrente> contasCorrentes = new Dictionary<string, ContaCorrente>() {
    { "André", new ContaCorrente("456") },
    { "João", new ContaCorrente("789") }
};


Boleto[] boletos = { Boleto.Criar("1", 450), Boleto.Criar("2", 150) , Boleto.Criar("3", 1500) };

BankFunctions bf = new BankFunctions(contaCorrentPrincipal, boletos, contasCorrentes);
bf.ToString()

In [ ]:
var tools = caller.GetDefinitionsFromObject(bf);
OpenAI.Chat.ChatCompletionOptions completionOptions = new()
{
    MaxOutputTokenCount=400,
    Temperature=0f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    TopP = 0.95f
};

foreach (var tool in tools)
{
    completionOptions.Tools.Add(tool);
}


In [ ]:
messages.Add(new OpenAI.Chat.UserChatMessage("Pague todos os boletos da minha conta que não estouram o saldo e transfira 100 para o João. Me diga o saldo."));



In [ ]:
do {
    OpenAI.Chat.ChatCompletion response = chatClient.CompleteChat(messages, completionOptions);
    // Include the message in the conversation history
    messages.Add(new AssistantChatMessage(response));
    if (response.FinishReason == OpenAI.Chat.ChatFinishReason.ToolCalls)
    {
        foreach (ChatToolCall toolCall in response.ToolCalls) {
            var functionReturn = caller.ExecuteTool(toolCall, bf);
            messages.Add(new ToolChatMessage(toolCall.Id, functionReturn));
        }
    } else {
        Console.WriteLine(response.Content[0].Text);
        break;
    }

}while(true);



In [ ]:
bf.ToString()